### Differentiation of tensor networks

This notebook describes examples on how to use differentiation to get the argmax of a tensornetwork. Each axis has to have only 2 dimensions!

In [17]:
import torch
import tensor

A Tensor can be created using a serialized form. 
1. tensor in serialized form:
<br>
- the value $(i_1, ... , i_k)$ in the Tensor is in place $\sum _{j=1} ^k i_j\cdot 2^{j-1}$ in the serialization
<br>
2. tensor with single axis:
<br>
- t = torch.tensor([a, b], dtype=torch.float32, requires_grad=True)
<br>
3. tensors with multiple axis:
<br>
- t = torch.tensor([serialized form of t])
- you dont need a gradient for this since only the derivatives of the single axis matter
<br>
4. reshaping:
<br>
- for every tensor, reshape it using the function tensor.get_torch_tensor(t, axis)
- axis is a list containing all axises of the tensor. If its a single axis k, just use [k]
- if you have multiple axis, use [k_1, ..., k_n]
<br>
5. tensornetwork:
<br>
- create the tensornetwork containing all reshaped tensors

In [3]:
t_1 = torch.tensor([0, 1], dtype=torch.float32, requires_grad=True)
x_1 = tensor.get_torch_tensor(t_1, [1])

t_2 = torch.tensor([0, 2], dtype=torch.float32, requires_grad=True)
x_2 = tensor.get_torch_tensor(t_2, [2])

t_3 = torch.tensor([0, 1], dtype=torch.float32, requires_grad=True)
x_3 = tensor.get_torch_tensor(t_3, [3])

t_12 = torch.tensor([0, 0, 0, -3])
x_12 = tensor.get_torch_tensor(t_12, [1, 2])

t_23 = torch.tensor([0, 0, 0, 2])
x_23 = tensor.get_torch_tensor(t_23, [2, 3])

t_13 = torch.tensor([0, 0, 0, 1])
x_13 = tensor.get_torch_tensor(t_13, [1, 3])

tn = [x_1, x_2, x_3, x_12, x_23, x_13]

Now we can combine the tensornetwork and aggregate it. To get the derivative just use 'aggregation.backward()'

In [4]:
c = tensor.combine(tn)
a = tensor.aggregate(c)

a.backward()

get the gradients of the single axis tensors

In [6]:
print(t_1.grad, t_2.grad, t_3.grad)

tensor([1., 0.]) tensor([0., 1.]) tensor([0., 1.])


### multiple Maxima

an example without a unique maximum. The algorithm chooses one configuration. The configuration seems to be always the same.

In [13]:
t_1 = torch.tensor([0, 1], dtype=torch.float, requires_grad=True)
t_2 = torch.tensor([0, 1], dtype=torch.float, requires_grad=True)
t_12 = torch.tensor([0, 0, 0, -1], dtype=torch.float, requires_grad=True)

x_1 = tensor.get_torch_tensor(t_1, [1])
x_2 = tensor.get_torch_tensor(t_2, [2])
x_12 = tensor.get_torch_tensor(t_12, [1, 2])

tn = [x_1, x_2, x_12]

In [14]:
c = tensor.combine(tn)
a = tensor.aggregate(c)

a.backward()

In [15]:
print(t_1.grad, t_2.grad)

tensor([0., 1.]) tensor([1., 0.])
